In [83]:
from graph_tool.all import *
import numpy as np
import pandas as pd
import matplotlib

In [ ]:
g = Graph()
v1 = g.add_vertex()
v2 = g.add_vertex()
e = g.add_edge(v1, v2)

In [ ]:
graph_draw(g, vertex_text=g.vertex_index)

In [ ]:
g = collection.data["celegansneural"]
print(g)

In [ ]:
state = minimize_nested_blockmodel_dl(g)
S1 = state.entropy()
mcmc_anneal(state, beta_range=(1, 10), niter=1000, mcmc_equilibrate_args=dict(force_niter=10))
state.entropy() - S1

In [ ]:
state.draw()

In [ ]:
g = load_graph("level7.gt")
g

In [ ]:
abun = pd.read_csv("level-7.csv", index_col="index")
otus = np.array([col for col in abun.columns if "k__" in col])
samples = np.array(abun.index)

In [ ]:
sp = [c for c in abun.columns if "k__" in c]
len([col for col in sp if sum(abun[col]) > 100])

In [ ]:
g = Graph(directed=False)
g.add_vertex(len(samples) + len(otus))

In [ ]:
name = g.vp.name = g.new_vertex_property("string", np.append(samples, otus))
part = g.vp.part = g.new_vertex_property("int", np.append(np.ones(len(samples)), np.zeros(len(otus))))
counts = g.ep.counts = g.new_edge_property("double")

In [ ]:
for si in range(len(samples)):
    # v = g.add_vertex()
    # name[v] = s
    for ti in range(len(samples), len(samples) + len(otus)):
        # w = g.add_vertex()
        c = int(abun.loc[name[si], name[ti]])
        if c > 0:
            e = g.add_edge(si, ti)
            counts[e] = np.log(c) + 1

In [ ]:
g.save("level7.gt")

## Fit the hSBM

In [ ]:
state_args = {"clabel": part, "pclabel": part, "eweight": counts, "deg_corr": True}
state_tmp = minimize_nested_blockmodel_dl(g, state_args=state_args)

In [ ]:
S1 = state_tmp.entropy()

for i in range(100): # this should be sufficiently large
    state_tmp.multiflip_mcmc_sweep(beta=np.inf, niter=10)

S2 = state_tmp.entropy()
print("Improvement:", S2 - S1)

In [ ]:
L = 0
for s in state_tmp.levels:
    L += 1
    if s.get_nonempty_B() == 2:
        break
state = state_tmp.copy(bs=state_tmp.get_bs()[:L] + [np.zeros(1)])

In [ ]:

state.draw(
    layout="bipartite", output="tmp.pdf", subsample_edges=1000, hshortcuts=1, 
    # edge_pen_width=prop_to_size(g.ep.counts, ma=4, power=1.2, log=False), 
    edge_pen_width=1.5,
    vertex_size=1.6,
    vertex_fill_color=part,
    vertex_color=part,
    edge_color=prop_to_size(counts, ma=10, power=1),
    ecmap=(cm.inferno, .4), edge_gradient=[], eorder=counts
)

In [ ]:
# levels = state.project_partition()
from collections import defaultdict
# group_id = nested_contiguous_map(state.get_bs())[1]
group_id = state.get_levels()[4].get_blocks().a
group_names = defaultdict(lambda: list())
for v, i in enumerate(group_id):
    group_names[i].append(name[v])

In [ ]:
membership = dict()
for lvl in range(4):
   state_proj = state.project_level(lvl)
   mem = defaultdict(lambda: list())
   for v, i in enumerate(state_proj.get_blocks().a):
      mem[i].append(name[v])
   membership[lvl] = list(mem.values())

In [ ]:
membership

In [ ]:
import pickle
with open("module-members.pickle", "wb") as f:
    pickle.dump(membership, f)

### Get two models ((not) degree corrected) using `seastar_hsbm` class

In [1]:
from seastar_hsbm_inference import seastar_hsbm

comm = seastar_hsbm()
comm.make_graph("level-7.csv")
comm.g
# comm.fit(deg_corr=False)

<Graph object, undirected, with 349 vertices and 3954 edges, 2 internal vertex properties, 1 internal edge property, at 0x7ff9c14a3f40>

In [84]:
# comm.save_model("hsbm-tmp-fit")
comm = seastar_hsbm()
comm.load_model("hsbm-fit2.pickle")
g = comm.state.g

In [75]:
ord = g.vp.ord = g.new_vertex_property("int", comm.state.project_level(2).get_blocks().a)
text = g.vp.text = g.new_vertex_property("string", comm.state.project_level(1).get_blocks().a)

In [81]:
part = comm.g.vp.part
counts = comm.g.ep.counts
blue = list(matplotlib.colors.ColorConverter().to_rgba("#729fcf"))
blue[-1] = 0.8

import graph_tool.all as gt
pos = gt.draw_hierarchy(
    comm.state,
    layout="bipartite", output="hsbm-fit-final.svg", subsample_edges=400, hshortcuts=1, 
    # edge_pen_width=prop_to_size(g.ep.counts, ma=4, power=1.2, log=False), 
    edge_pen_width=1.5,
    vertex_size=1.8,
    hvertex_size = 6.5,
    # vertex_fill_color=[0.6, 0.6, 0.6, 0.7],
    # vertex_color=[0.6, 0.6, 0.6, 0.7],
    vertex_fill_color=ord,
    vertex_color=ord,
    hvertex_color=[1, 1, 1, 0.4],
    hedge_color = blue,
    hvertex_fill_color = blue,
    # vertex_text=text,
    # vertex_text_offset=[-0.05, 0.0],
    edge_color=gt.prop_to_size(counts, ma=10, power=1),
    ecmap=(matplotlib.cm.inferno, 0.4), edge_gradient=[], eorder=counts
)

In [91]:
g = comm.state.get_bstack()[0]
modularity(g, g.vp.b, weight=g.ep.count)

-0.017728604567703206

In [ ]:
from collections import defaultdict
membership = dict()
for lvl in range(len(comm.state.levels) - 1):
    state_proj = comm.state.project_level(lvl)
    mem = defaultdict(lambda: list())
    for v, i in enumerate(state_proj.get_blocks().a):
        mem[str(i)].append(comm.g.vp.name[v])
    membership[lvl] = dict(mem)

In [ ]:
import pickle
with open("tmp.pickle", 'wb') as f:
    pickle.dump(membership, f)